# Data preparation for Final dataset

In [1]:
import os
os.chdir("../../")

In [2]:
import exoNet
import pandas as pd
import numpy as np

Using TensorFlow backend.


In [3]:
data_name = "final"

In [5]:
data = pd.read_csv(f'./Data/{data_name}/{data_name}.design.mat.shortened.csv', index_col='id')
data.head()

,seq,dotbracket,element_string,element_string_number
id,,,,
chr1_10140_10163_+_ev,UGGGAUUGGGGAUUGGGAUUGGGA,........................,ffffffffffffffffffffffff,0
chr1_10320_10341_+_ev,GGUUGGGAUUGGGGAUUGGGAU,......................,ffffffffffffffffffffff,0
chr1_10381_10400_+_ev,AUUGGGAUUGGGGAUUGGGG,....................,ffffffffffffffffffff,0
chr1_10496_10524_+_ev,GGCCCAGACUGGACUCCUCUUGACACGAG,((.(((...)))...))............,ssissshhhsssiiisstttttttttttt,11100011100000000262799360
chr1_10749_10778_+_ev,GCGGCGCGGCCGCGUCCGCGUCUCUCCGCG,(((((((((......))))).....)))).,ssssssssshhhhhhsssssiiiiisssst,11111000000111111032537088


In [6]:
data_columns = data.columns

In [7]:
seq_columns = ['seq', 'dotbracket', 'element_string', 'element_string_number']

In [8]:
seq_data_df = data[seq_columns]

In [9]:
labels = pd.read_csv(f'./Data/{data_name}/{data_name}.label.mat.csv')
labels.head()

,label
0,YES
1,YES
2,YES
3,YES
4,YES


In [10]:
np.save(file=f"./Data/{data_name}/labels.npy", arr=labels)

In [11]:
seq_data = seq_data_df['seq'].values
seq_data

array(['UGGGAUUGGGGAUUGGGAUUGGGA', 'GGUUGGGAUUGGGGAUUGGGAU',
       'AUUGGGAUUGGGGAUUGGGG', ..., 'ACUACAAAGCUUCAUAUUA',
       'GAGACCAACAUUCUUUCUCUCA', 'UAGAGCUGAAGUGGUACAUUCA'], dtype=object)

In [12]:
char_encoder = {
    'N': -1,
    'A': 0,
    'C': 1,
    'G': 2,
    'U': 3
}

In [14]:
seq_encoded = exoNet.pep.seq_encoder(seq_data, char_encoder, 50, unknown_char=True)
seq_encoded.shape

(80841, 50, 4)

In [15]:
db_encoder = {
    '.': 0,
    ')': 1,
    '(': 2,
}

In [17]:
db_encoded = seq_data_df['dotbracket'].values
db_encoded = exoNet.pep.seq_encoder(db_encoded, db_encoder, 50, unknown_char=False)
db_encoded.shape

(80841, 50, 3)

In [18]:
element_string_encoder = {
    'f': 0,
    't': 1,
    's': 2,
    'i': 3,
    'm': 4,
    'h': 5,
}

In [19]:
element_string_encoded = seq_data_df['element_string'].values
element_string_encoded = exoNet.pep.seq_encoder(element_string_encoded, element_string_encoder, 50, unknown_char=False)
element_string_encoded.shape

(80841, 50, 6)

In [20]:
element_string_number_encoder = {
    '0': 0,
    '1': 1,
    '2': 2,
    '3': 3,
    '4': 4,
    '5': 5,
    '6': 6,
    '7': 7,
    '8': 8,
    '9': 9,
}

In [21]:
element_string_number_encoded = seq_data_df['element_string_number'].values
element_string_number_encoded = exoNet.pep.seq_encoder(element_string_number_encoded, element_string_number_encoder, 50, unknown_char=False)
element_string_number_encoded.shape

(80841, 50, 10)

In [22]:
seq_encoded = np.concatenate([seq_encoded, db_encoded, element_string_encoded, element_string_number_encoded], axis=2)
seq_encoded.shape

(80841, 50, 23)

In [23]:
np.save(file=f"./Data/{data_name}/sequences.npy", arr=seq_encoded)